# Text Classification with and without Pre-Processing

## Srinitish Srinivasan
## 21BAI1394


In [27]:
import pandas as pd 
import numpy as np 
import nltk 
from nltk.tokenize import word_tokenize 

from nltk.corpus import stopwords,wordnet
from nltk.stem import SnowballStemmer,WordNetLemmatizer

nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")
nltk.download("stopwords")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,f1_score,accuracy_score,confusion_matrix,roc_curve,auc,roc_auc_score



[nltk_data] Downloading package punkt to /Users/smudge/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/smudge/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/smudge/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/smudge/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
#Get the dataset
#Spam classification Dataset from UCI Repository

import os 
from dotenv import load_dotenv

load_dotenv('.env')
path=os.getenv("spam_classification")

dataset=pd.read_csv(path,encoding='ISO-8859-1')
dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# Text Pre-processing followed by Classification
## Datset: UCI Spam classification

In [29]:
#Remove punctuations
import re
import string

def preprocess(text):
    text=text.lower()
    text=text.strip()

    text=re.compile('<.&?').sub('',text)
    text=re.compile('[%s]'% re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 

    return text

def stopword(text):
    a=[i for i in text.split() if i not in stopwords.words('english')]

    return ' '.join(a)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
wl=WordNetLemmatizer()

def lemmatizer(word):
    word_pos_tags=nltk.pos_tag(word_tokenize(word))

    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)]
    return " ".join(a)



<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
/var/folders/m7/fg2s2g0x5tdfqnlsn52yrt3w0000gn/T/ipykernel_51836/1518927956.py:11: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)


In [30]:
#Preprocess the text

def total_preprocess(text):
    return lemmatizer(stopword(preprocess(text)))

dataset['v2']=dataset['v2'].apply(lambda x:total_preprocess(x))

dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,go jurong point crazy available bugis n great ...,NaN,NaN,NaN
1,ham,ok lar joking wif u oni,NaN,NaN,NaN
2,spam,free entry wkly comp win fa cup final tkts st ...,NaN,NaN,NaN
3,ham,u dun say early hor u c already say,NaN,NaN,NaN
4,ham,nah think go usf life around though,NaN,NaN,NaN


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Map Target train and test to 0 and 1 from no spam and spam
mapping={
    'spam':1,
    'ham':0
}

dataset['v1']=dataset['v1'].map(mapping)

X_train,X_test,Y_train,Y_test=train_test_split(dataset['v2'],dataset['v1'],test_size=0.20,shuffle=True)

#Inverse document frequency based featurisation
idf=TfidfVectorizer(use_idf=True)
X_train_vectors=idf.fit_transform(X_train)
X_test_vectors=idf.transform(X_test)

dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,go jurong point crazy available bugis n great ...,NaN,NaN,NaN
1,0,ok lar joking wif u oni,NaN,NaN,NaN
2,1,free entry wkly comp win fa cup final tkts st ...,NaN,NaN,NaN
3,0,u dun say early hor u c already say,NaN,NaN,NaN
4,0,nah think go usf life around though,NaN,NaN,NaN


In [32]:
logistic_reg=LogisticRegression(solver='liblinear',C=10,penalty='l2')
logistic_reg.fit(X_train_vectors,Y_train)


LogisticRegression(C=10, solver='liblinear')

## Results with Text Pre-processing

In [33]:
y_pred=logistic_reg.predict(X_test_vectors)
y_prb=logistic_reg.predict_proba(X_test_vectors)[:,1]

print("Results with Text Pre-Processing")
print(classification_report(Y_test,y_pred))

Results with Text Pre-Processing
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.99      0.88      0.93       149

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [34]:
fpr,tpr,_=roc_curve(Y_test,y_prb)
roc_auc=auc(fpr,tpr)
roc_auc

0.9907040726999875

# No pre-processing before text classification
## Dataset: UCI Spam Classification

In [35]:
#Get the dataset
#Spam classification Dataset from UCI Repository

import os 
from dotenv import load_dotenv

load_dotenv('.env')
path=os.getenv("spam_classification")

dataset=pd.read_csv(path,encoding='ISO-8859-1')
dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Map Target train and test to 0 and 1 from no spam and spam
mapping={
    'spam':1,
    'ham':0
}

dataset['v1']=dataset['v1'].map(mapping)

X_train,X_test,Y_train,Y_test=train_test_split(dataset['v2'],dataset['v1'],test_size=0.20,shuffle=True)

#Inverse document frequency based featurisation
idf=TfidfVectorizer(use_idf=True)
X_train_vectors=idf.fit_transform(X_train)
X_test_vectors=idf.transform(X_test)

dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [37]:
logistic_reg=LogisticRegression(solver='liblinear',C=10,penalty='l2')
logistic_reg.fit(X_train_vectors,Y_train)


LogisticRegression(C=10, solver='liblinear')

## Results without Text Pre-Processing

In [38]:
y_pred=logistic_reg.predict(X_test_vectors)
y_prb=logistic_reg.predict_proba(X_test_vectors)[:,1]

print("Results without Text Pre-Processing")
print(classification_report(Y_test,y_pred))

Results without Text Pre-Processing
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       950
           1       0.98      0.91      0.94       165

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.97      1115
weighted avg       0.98      0.98      0.98      1115



In [39]:
fpr,tpr,_=roc_curve(Y_test,y_prb)
roc_auc=auc(fpr,tpr)
roc_auc

0.9875406698564593